# 1 Introdução

Primeiramente, vamos separar uma porcentagem do conjunto de treino para testar o modelo e poder ter uma métrica. O objetivo da v2 será atingir uma métrica melhor.

# 2 Libs

In [39]:
import pandas as pd

import nltk
from string import punctuation
from nltk import tokenize
import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns

# 3 Carrega dataset e separa conjunto de teste

In [13]:
df_treino = pd.read_csv('tweets_treino.csv',sep=';')
df_treino.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


In [14]:
df_treino.shape

(100000, 5)

In [15]:
df_teste = df_treino.sample(frac=.1,random_state=42)
df_teste.head()

,id,tweet_text,tweet_date,sentiment,query_used
75721,1045103224259325952,Casagrande fala de esporte e política: 'O fute...,Thu Sep 27 00:10:00 +0000 2018,2,estadao
80184,1042126369130463232,Juízes distantes dos tribunais de origem https...,Tue Sep 18 19:01:02 +0000 2018,2,folha
19864,1046797258392571904,@mynameiskb24 Ah ! Lucas content :),Mon Oct 01 16:21:29 +0000 2018,1,:)
76699,1044579206720618496,Claudia Leitte faz campanha de suplemento para...,Tue Sep 25 13:27:44 +0000 2018,2,exame
92991,1039121256816226304,Uma semana cheia de boas atitudes! 🥇 #motivaçã...,Mon Sep 10 11:59:48 +0000 2018,2,#oportunidade


In [16]:
df_teste.shape

(10000, 5)

In [17]:
df_treino = df_treino.drop(df_teste.index)
df_treino.shape

(90000, 5)

# 4 Tratamento

## 4.1 Cria coluna de classificação

In [18]:
df_treino = df_treino.dropna(subset=['tweet_text', 'sentiment'])

In [19]:
df_treino = df_treino[['tweet_text', 'tweet_date', 'sentiment']]

In [20]:
df_treino['sentiment'].value_counts()

2    30033
0    30019
1    29948
Name: sentiment, dtype: int64

In [21]:
df_treino['classificacao'] = df_treino['sentiment'].apply(lambda x : 'Positivo' if x == 1 else 'Neutro' if x == 2 else 'Negativo')

In [22]:
df_teste = df_teste.dropna(subset=['tweet_text','sentiment'])

In [23]:
df_teste = df_teste[['tweet_text', 'tweet_date', 'sentiment']]

In [24]:
df_teste['classificacao'] = df_teste['sentiment'].apply(lambda x : 'Positivo' if x == 1 else 'Neutro' if x == 2 else 'Negativo')

In [25]:
df_treino.head()

,tweet_text,tweet_date,sentiment,classificacao
0,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,Positivo
1,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,Positivo
2,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,Positivo
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,Positivo
4,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,Positivo


In [26]:
df_teste.head()

,tweet_text,tweet_date,sentiment,classificacao
75721,Casagrande fala de esporte e política: 'O fute...,Thu Sep 27 00:10:00 +0000 2018,2,Neutro
80184,Juízes distantes dos tribunais de origem https...,Tue Sep 18 19:01:02 +0000 2018,2,Neutro
19864,@mynameiskb24 Ah ! Lucas content :),Mon Oct 01 16:21:29 +0000 2018,1,Positivo
76699,Claudia Leitte faz campanha de suplemento para...,Tue Sep 25 13:27:44 +0000 2018,2,Neutro
92991,Uma semana cheia de boas atitudes! 🥇 #motivaçã...,Mon Sep 10 11:59:48 +0000 2018,2,Neutro


# 5 Processamento do texto

Principais funções:

1) Stopwords - elimina palavras irrelevantes para a análise semântica (e, ou, a, o, as, os).
2) Punctuation - elimina pontos e símbolos (.'!"*&$%).
3) Tokenizer - quebra o texto em 'tokens' (partes), para que possa ser compreendido palavra por palavra.
4) Unidecode - retira os acentos.
5) Lower - deixa todas as letras em minúsculo.
6) Stemmer - utiliza somente os radicais das palavras, facilitando a interpretação de uma palavra que tenha o mesmo valor semântico, mesmo que em diferentes tempos verbais, gêneros ou número.

In [27]:
def tratar(df, coluna):

  palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
  pontuacao = []
  for ponto in punctuation:
    pontuacao.append(ponto)
  pontuacao_stopwords = pontuacao + palavras_irrelevantes
  token_pontuacao = tokenize.WordPunctTokenizer()
  sem_acentos = [unidecode.unidecode(texto) for texto in df[coluna]]
  stopwords_sem_acentos = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]
  stemmer = nltk.RSLPStemmer()

  df['texto_tratado'] = sem_acentos

  frase_processada = []

  for texto in df['texto_tratado']:
    
    nova_frase = []
    texto = texto.lower()
    texto = texto.replace('rt ','')
    texto = texto.replace('http','')
    texto = texto.replace('...','')
    texto = texto.replace('://','')
    texto = texto.replace('..','')
    palavras_texto = token_pontuacao.tokenize(texto)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))

  df['texto_tratado'] = frase_processada

In [28]:
tratar(df_treino,'tweet_text')

In [29]:
tratar(df_teste,'tweet_text')

In [30]:
df_treino.sample(5)

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado
47850,@XXKWY o meu pc quebrou :( não tenho mais lol,Tue Oct 02 03:27:53 +0000 2018,0,Negativo,xxkwy pc quebr :( nao lol
67943,"Líderes, PT e PSDB perdem mais de 30% dos voto...",Wed Oct 10 11:08:33 +0000 2018,2,Neutro,lid pt psdb perd 30 vot sen psl disp fic 3o lu...
14158,@ccIncredible de nada :),Wed Oct 03 14:53:19 +0000 2018,1,Positivo,ccincredibl nad :)
23000,@ap_fernandes @marimessias @modices @manubarem...,Mon Oct 01 11:41:34 +0000 2018,1,Positivo,ap_fernand marimess modic manub jaridarra gost...
893,✨ PROMOÇÃO: COMBINADOS | O Pico Espeto Bar ✨ 3...,Fri Oct 12 15:00:06 +0000 2018,1,Positivo,promoca combin pic espet bar 3 espet 1 cervej ...


In [31]:
df_teste.sample(5)

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado
91237,Semana Nacional do Trânsito. Educação e Consci...,Tue Sep 18 11:27:40 +0000 2018,2,Neutro,seman nacion transit educaca conscientizaca jo...
92908,Isso sem contar as estrelas que formam o logo ...,Mon Sep 10 16:05:08 +0000 2018,2,Neutro,cont estrel form log rockst carr fantasm pe gr...
56838,vou sentir tanta saudade :( #MPN #JoaoGuilherm...,Mon Oct 01 15:21:54 +0000 2018,0,Negativo,vou sent tant saudad :( mpn joaoguilherm st co...
4145,"Espero que gostem de câmeras, pois ele quer ed...",Mon Oct 08 13:27:21 +0000 2018,1,Positivo,esper gost cam poi qu educaca distanc :) st co...
28486,@mthware eu queria fazer o meu almoço :),Thu Sep 27 21:08:07 +0000 2018,1,Positivo,mthw quer faz almoc :)


# 6 Teste

In [32]:
regressao_logistica = LogisticRegression()

In [33]:
def classificar(df, texto, sentimento, df_teste, texto_teste):
  tfidf = TfidfVectorizer(lowercase=False,ngram_range=(1,2))

  treino_tfidf = tfidf.fit_transform(df[texto])
  teste_tfidf = tfidf.transform(df_teste[texto_teste]).todense()

  treino = treino_tfidf
  classe_treino = df[sentimento]
  teste = teste_tfidf

  regressao_logistica.fit(treino,classe_treino)

  classificacao = regressao_logistica.predict(teste)

  df_teste['sentimento'] = classificacao

In [34]:
df_teste

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado
75721,Casagrande fala de esporte e política: 'O fute...,Thu Sep 27 00:10:00 +0000 2018,2,Neutro,casagrand fal esport poli futebol fic egoist c...
80184,Juízes distantes dos tribunais de origem https...,Tue Sep 18 19:01:02 +0000 2018,2,Neutro,juiz dist tribun orig st co ar9rh8f8e4
19864,@mynameiskb24 Ah ! Lucas content :),Mon Oct 01 16:21:29 +0000 2018,1,Positivo,mynameiskb24 ah luc content :)
76699,Claudia Leitte faz campanha de suplemento para...,Tue Sep 25 13:27:44 +0000 2018,2,Neutro,claud leitt faz campanh suplement intoleranc l...
92991,Uma semana cheia de boas atitudes! 🥇 #motivaçã...,Mon Sep 10 11:59:48 +0000 2018,2,Neutro,seman chei boa atitud motivaca sucess oportun ...
...,...,...,...,...,...
5002,"@lopes85 @_Goalpoint ""gentes"" do benfiquistão ...",Mon Oct 08 11:52:35 +0000 2018,1,Positivo,lopes85 _goalpoint gent benfiquista nao gost f...
30151,@DeFerrazdede @srta_quitete @xquadrado @BrunoT...,Thu Sep 27 19:18:55 +0000 2018,1,Positivo,deferrazded srta_quitet xquadr brunotdd aosnet...
93194,"Vaga para Técnico de Enfermagem do Trabalho, e...",Sun Sep 09 01:40:45 +0000 2018,2,Neutro,vag tecn enferm trabalh farroupilh rs sucess v...
73199,"&gt;@EstadaoPolitica Neutro no 1º turno, PSB v...",Mon Oct 01 22:40:00 +0000 2018,2,Neutro,gt ;@ estadaopoli neutr 1o turn psb vai posic ...


In [35]:
classificar(df_treino, 'texto_tratado', 'classificacao', df_teste, 'texto_tratado')

c:\Users\Wesley Ribeiro\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
df_treino

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado
0,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,Positivo,laranjito76 pesso cert val azeved :)
1,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,Positivo,behin_d_curtain mim precis contrari :) vem chu...
2,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,Positivo,vou faz vide hoj pens fal pouc sobr nov met cs...
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,Positivo,aaaaaaa ame tant polaroid sei express quant to...
4,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,Positivo,valoriz coraca menin vc difer fac sorr ter cer...
...,...,...,...,...,...
99995,Confira esta vaga na Senac-RS: Editor De Video...,Fri Aug 17 14:57:05 +0000 2018,2,Neutro,conf vag senac rs edit vide port alegr rs suce...
99996,Confira esta vaga na Grupo SABEMI: Assistente ...,Fri Aug 17 14:55:26 +0000 2018,2,Neutro,conf vag grup sabem assist administraca pessoa...
99997,Confira esta vaga na Groupe L’OCCITANE: Gerent...,Fri Aug 17 14:52:48 +0000 2018,2,Neutro,conf vag group l occitan gerent distrit sul po...
99998,"Está no ar o CandidatoWEB, seu novo portal de ...",Fri Aug 17 14:50:33 +0000 2018,2,Neutro,ar candidatoweb nov port busc vag empreg vagas...


In [36]:
df_teste

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado,sentimento
75721,Casagrande fala de esporte e política: 'O fute...,Thu Sep 27 00:10:00 +0000 2018,2,Neutro,casagrand fal esport poli futebol fic egoist c...,Neutro
80184,Juízes distantes dos tribunais de origem https...,Tue Sep 18 19:01:02 +0000 2018,2,Neutro,juiz dist tribun orig st co ar9rh8f8e4,Neutro
19864,@mynameiskb24 Ah ! Lucas content :),Mon Oct 01 16:21:29 +0000 2018,1,Positivo,mynameiskb24 ah luc content :),Negativo
76699,Claudia Leitte faz campanha de suplemento para...,Tue Sep 25 13:27:44 +0000 2018,2,Neutro,claud leitt faz campanh suplement intoleranc l...,Neutro
92991,Uma semana cheia de boas atitudes! 🥇 #motivaçã...,Mon Sep 10 11:59:48 +0000 2018,2,Neutro,seman chei boa atitud motivaca sucess oportun ...,Neutro
...,...,...,...,...,...,...
5002,"@lopes85 @_Goalpoint ""gentes"" do benfiquistão ...",Mon Oct 08 11:52:35 +0000 2018,1,Positivo,lopes85 _goalpoint gent benfiquista nao gost f...,Positivo
30151,@DeFerrazdede @srta_quitete @xquadrado @BrunoT...,Thu Sep 27 19:18:55 +0000 2018,1,Positivo,deferrazded srta_quitet xquadr brunotdd aosnet...,Positivo
93194,"Vaga para Técnico de Enfermagem do Trabalho, e...",Sun Sep 09 01:40:45 +0000 2018,2,Neutro,vag tecn enferm trabalh farroupilh rs sucess v...,Neutro
73199,"&gt;@EstadaoPolitica Neutro no 1º turno, PSB v...",Mon Oct 01 22:40:00 +0000 2018,2,Neutro,gt ;@ estadaopoli neutr 1o turn psb vai posic ...,Neutro


In [41]:
df_teste['validacao'] = df_teste['classificacao'] == df_teste['sentimento']
df_teste['validacao'] = df_teste['validacao'].astype(int)
df_teste

,tweet_text,tweet_date,sentiment,classificacao,texto_tratado,sentimento,validacao
75721,Casagrande fala de esporte e política: 'O fute...,Thu Sep 27 00:10:00 +0000 2018,2,Neutro,casagrand fal esport poli futebol fic egoist c...,Neutro,1
80184,Juízes distantes dos tribunais de origem https...,Tue Sep 18 19:01:02 +0000 2018,2,Neutro,juiz dist tribun orig st co ar9rh8f8e4,Neutro,1
19864,@mynameiskb24 Ah ! Lucas content :),Mon Oct 01 16:21:29 +0000 2018,1,Positivo,mynameiskb24 ah luc content :),Negativo,0
76699,Claudia Leitte faz campanha de suplemento para...,Tue Sep 25 13:27:44 +0000 2018,2,Neutro,claud leitt faz campanh suplement intoleranc l...,Neutro,1
92991,Uma semana cheia de boas atitudes! 🥇 #motivaçã...,Mon Sep 10 11:59:48 +0000 2018,2,Neutro,seman chei boa atitud motivaca sucess oportun ...,Neutro,1
...,...,...,...,...,...,...,...
5002,"@lopes85 @_Goalpoint ""gentes"" do benfiquistão ...",Mon Oct 08 11:52:35 +0000 2018,1,Positivo,lopes85 _goalpoint gent benfiquista nao gost f...,Positivo,1
30151,@DeFerrazdede @srta_quitete @xquadrado @BrunoT...,Thu Sep 27 19:18:55 +0000 2018,1,Positivo,deferrazded srta_quitet xquadr brunotdd aosnet...,Positivo,1
93194,"Vaga para Técnico de Enfermagem do Trabalho, e...",Sun Sep 09 01:40:45 +0000 2018,2,Neutro,vag tecn enferm trabalh farroupilh rs sucess v...,Neutro,1
73199,"&gt;@EstadaoPolitica Neutro no 1º turno, PSB v...",Mon Oct 01 22:40:00 +0000 2018,2,Neutro,gt ;@ estadaopoli neutr 1o turn psb vai posic ...,Neutro,1


In [42]:
len(df_teste[df_teste['validacao']==1])

8026

In [43]:
accuracy = accuracy_score(df_teste['classificacao'], df_teste['sentimento'])
accuracy

0.8026

In [40]:
b_accuracy = balanced_accuracy_score(df_teste['classificacao'], df_teste['sentimento'])
b_accuracy

0.8036604018210127